In [8]:
import pandas as pd
import numpy as np
import gym
from gym import spaces
from stable_baselines3 import PPO
from gym.envs.registration import register
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
# Load datasets
diets = pd.read_csv('diets.csv')
medications = pd.read_csv('medications.csv')
precautions = pd.read_csv('precautions_df.csv')
symptom_severity = pd.read_csv('Symptom-severity.csv')
symptoms = pd.read_csv('symtoms_df.csv')
workouts = pd.read_csv('workout_df.csv')

In [3]:
# Prepare the symptoms dataset
symptoms['symptoms'] = symptoms[['Symptom_1', 'Symptom_2', 'Symptom_3', 'Symptom_4']].apply(
    lambda x: [s.strip() for s in x.tolist() if isinstance(s, str)], axis=1
)

In [4]:
# One-hot encode symptoms
mlb = MultiLabelBinarizer()
symptom_encoded = mlb.fit_transform(symptoms['symptoms'])
encoded_symptoms_df = pd.DataFrame(symptom_encoded, columns=mlb.classes_)

In [5]:
# Concatenate encoded symptoms with diseases
final_data = pd.concat([symptoms[['Disease']], encoded_symptoms_df], axis=1)

In [9]:
class HealthcareEnv(gym.Env):
    def __init__(self):
        super(HealthcareEnv, self).__init__()
        self.action_space = spaces.Discrete(len(diets) + len(medications) + len(workouts))
        self.observation_space = spaces.Box(low=0, high=1, shape=(encoded_symptoms_df.shape[1],), dtype=np.float32)
        self.state = None
        self.done = False

    def reset(self):
        # Randomly sample initial state
        self.state = encoded_symptoms_df.sample(1).values.flatten()
        self.done = False
        return np.array(self.state, dtype=np.float32)

    def step(self, action):
        # Reduce symptom severity based on action
        if action < len(diets):  # Diet action
            self.state = np.clip(self.state - np.random.rand(len(self.state)) * 0.1, 0, 1)
        elif action < len(diets) + len(medications):  # Medication action
            self.state = np.clip(self.state - np.random.rand(len(self.state)) * 0.2, 0, 1)
        else:  # Workout action
            self.state = np.clip(self.state - np.random.rand(len(self.state)) * 0.15, 0, 1)

        # Reward based on reduction in symptom severity
        reward = -np.sum(self.state)

        # Episode ends when all symptoms are reduced below a threshold
        self.done = np.all(self.state < 0.1)

        return np.array(self.state, dtype=np.float32), reward, self.done, {}

    def render(self, mode='human'):
        print(f"Current symptom severities: {self.state}")


In [10]:
# Register the environment
register(
    id='HealthcareEnv-v0',  # Unique identifier for your environment
    entry_point=__name__ + ':HealthcareEnv',  # Specify the path to your environment class
)

In [11]:
# Create environment
env = gym.make('HealthcareEnv')

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gym/envs/registration.py:563: UserWarning: WARN: Using the latest versioned environment `HealthcareEnv-v0` instead of the unversioned environment `HealthcareEnv`.
  logger.warn(


In [12]:
# Train the PPO model
model = PPO('MlpPolicy', env, verbose=1)
model.learn(total_timesteps=10000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gym/utils/passive_env_checker.py:174: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gym/utils/passive_env_checker.py:187: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.warn(


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 14.8     |
|    ep_rew_mean     | -24.9    |
| time/              |          |
|    fps             | 5688     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 14.7        |
|    ep_rew_mean          | -24.7       |
| time/                   |             |
|    fps                  | 3292        |
|    iterations           | 2           |
|    time_elapsed         | 1           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.014119925 |
|    clip_fraction        | 0.0731      |
|    clip_range           | 0.2         |
|    entropy_loss         | -6.2        |
|    explained_variance   | 0.00758     |
|    learning_rate        | 0.

In [13]:
# Save the model
model.save("healthcare_recommendation_model2")

In [14]:
# Load the trained model
model = PPO.load("healthcare_recommendation_model2")

In [15]:
# Define action mapping
action_mapping = { 
    **{i: f"Diet: {diets.iloc[i, 0]}" for i in range(len(diets))},
    **{len(diets) + i: f"Medication: {medications.iloc[i, 0]}" for i in range(len(medications))},
    **{len(diets) + len(medications) + i: f"Workout: {workouts.iloc[i, 0]}" for i in range(len(workouts))}
}

In [23]:
def get_recommendations(symptom_list, model, env, mlb, action_mapping):
    # One-hot encode user symptoms
    encoded_input = mlb.transform([symptom_list]).flatten()

    # Reset environment and set the state to user input
    env.reset()
    env.state = encoded_input
    env.done = False

    # Get recommendations
    state = np.array(encoded_input, dtype=np.float32)
    recommendations = []

    print("Processing your symptoms...")
    while True:
        # Predict action
        action, _ = model.predict(state)
        action = int(action)  # Convert action from NumPy array to scalar

        # Map action to recommendation
        recommendation = action_mapping.get(action, "Unknown Action")
        recommendations.append(recommendation)

        # Take the action in the environment
        next_state, reward, done, _ = env.step(action)
        if done:
            break
        state = next_state

    return recommendations


In [24]:
# Example: Get user input and provide recommendations
user_symptoms = input("Enter your symptoms separated by commas: ").split(',')
user_symptoms = [s.strip() for s in user_symptoms]

In [25]:
recommendations = get_recommendations(user_symptoms, model, env, mlb, action_mapping)

Processing your symptoms...


In [26]:
print("Here are your personalized recommendations:")
for rec in recommendations:
    print(f"- {rec}")

Here are your personalized recommendations:
- Workout: 7
- Workout: 42
- Workout: 318
- Workout: 225
- Diet: (vertigo) Paroymsal Positional Vertigo
- Workout: 334
- Workout: 72
- Workout: 165
- Workout: 54
- Workout: 9
- Workout: 35
- Workout: 132
- Workout: 342
